<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [9]</a>'.</span>

In [1]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [2]:
# url para o cojunto de dados 
url = "https://servicodados.ibge.gov.br/api/v3/agregados/1839/periodos/2022/variaveis/630?localidades=N1[all]"

In [3]:
spark = SparkSession.builder.getOrCreate()

/home/airflow/.local/lib/python3.11/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
response = requests.get(url=url)
response.raise_for_status()

In [5]:
data = response.json()

In [6]:
schema = T.StructType([
    T.StructField("id", T.StringType(), True),
    T.StructField("variavel", T.StringType(), True),
    T.StructField("unidade", T.StringType(), True),
    T.StructField("resultados", T.ArrayType(T.StructType([
        T.StructField("classificacoes", T.ArrayType(T.StructType([
            T.StructField("id", T.StringType(), True),
            T.StructField("nome", T.StringType(), True),
            T.StructField("categoria", T.MapType(T.StringType(), T.StringType()), True)
        ])), True),
        T.StructField("series", T.ArrayType(T.StructType([
            T.StructField("localidade", T.StructType([
                T.StructField("id", T.StringType(), True),
                T.StructField("nivel", T.StructType([
                    T.StructField("id", T.StringType(), True),
                    T.StructField("nome", T.StringType(), True)
                ]), True),
                T.StructField("nome", T.StringType(), True)
            ]), True),
            T.StructField("serie", T.MapType(T.StringType(), T.StringType()), True)
        ])), True)
    ])), True)
])

In [7]:
df = spark.createDataFrame(data, schema)

In [8]:
df_out = df.select(
    F.col("id"),
    F.col("variavel")
)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [9]:
df_out = df.select(
.show()

SyntaxError: '(' was never closed (2260856980.py, line 1)